In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import KFold
from skmultilearn.model_selection import iterative_train_test_split
from sklearn.metrics import f1_score,  precision_score, recall_score, hamming_loss

import openai
import json

## **Load Cleaned Data**

In [2]:
# Load the cleaned datasets
df_user_inputs = pd.read_csv('../dataset/user_inputs_cleaned.csv')
df_labels = pd.read_csv('../dataset//labels_cleaned.csv')

# Remove unnecessary index columns
df_user_inputs.drop(df_user_inputs.columns[0], axis=1, inplace=True)
df_labels.drop(df_labels.columns[0], axis=1, inplace=True)

# Ensure alignment
assert len(df_labels) == len(df_user_inputs), "Datasets do not align!"

print(df_user_inputs.shape)
df_user_inputs.head(10)

(3974, 1)


,text
0,er is een teek op mijn been ik ben bang dat di...
1,er is een teek op mijn rug en ik krijg hem er ...
2,op mijn been zit een teek ik heb hem geprobeer...
3,ik heb allergieen
4,huid
5,roodheid
6,schilfering
7,ik heb wratten onder mijn voet
8,ik heb gisteren naar het bos geweest en zie nu...
9,ik voelde iets prikken


In [3]:
df_labels.head()

,"Niet lekker voelen, algehele malaise",Beenklachten,Bloedneus,Misselijkheid en overgeven,Brandwond,Buikpijn,Suikerziekte (ontregeld),Diarree,Duizeligheid,Gebitsklachten,...,Coronavirus,Knieklachten,Liesklachten,Elleboogklachten,Schouderklachten,Oorsuizen,Hand- en polsklachten,Enkelklachten,Dikke enkels of voeten,Vingerklachten
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
# Determine model baseline
label_frequencies = df_labels.sum().sort_values(ascending=False)
label_frequencies / df_labels.shape[0]

Huidklachten                            0.088827
Beenklachten                            0.072723
Buikpijn                                0.062154
Oorklachten                             0.052843
Misselijkheid en overgeven              0.037242
                                          ...   
Liesklachten                            0.005033
Tekenbeet                               0.005033
Verdrinking                             0.005033
Verwonding aan de buik                  0.005033
Niet lekker voelen, algehele malaise    0.002516
Length: 74, dtype: float64

We see that if we predict every time the label with the highest frequency (Huidklachten), our model will be correct around 9% of the time. We want our model to perform at least better than this 9% threshold

## **Prepare Data to Model Format**

To accomodate the nature of multi-label classification. Instead of using the traditional method `train_test_split`, we employ iterative stratified sampling `iterative_train_test_split`, to provide a well-balanced distribution of all label combinations in both training and test sets.

In [5]:
## Split data to train:val:test

# Prepare data for iterative train test split
# X must be 2D np.ndarray and y must be 2D binary np.ndarray
X_texts = df_user_inputs['text'].values
X_texts = X_texts.reshape(-1, 1)
y = df_labels.values

# Split the data 60:20:20 with multi-label stratification
train_texts, y_train, test_texts, y_test = iterative_train_test_split(X_texts, y, test_size = 0.2)
#val_texts, y_val, test_texts, y_test = iterative_train_test_split(tmp_texts, y_tmp, test_size = 0.5)

# Sanity checks to confirm the shapes of the datasets
assert train_texts.shape[0] == y_train.shape[0], "Mismatch in train data and labels"
assert test_texts.shape[0] == y_test.shape[0], "Mismatch in test data and labels"

train_texts, test_texts = train_texts.ravel(), test_texts.ravel()
#val_texts = val_texts.ravel()

print(train_texts.shape, y_train.shape, test_texts.shape)
train_texts

(3175,) (3175, 74) (799,)


array(['er is een teek op mijn been ik ben bang dat die er al een tijdje op heeft gezeten',
       'huid', 'roodheid', ...,
       'vannacht met slapen denk ik gekke beweging gemaakt want mn nek is nu helemaal stijf kan niet meer naar rechts kijken',
       'heb al langere tijd pijn in mn nek krijg dan soms tintelingen over mijn arm heb dan ook minder kracht in mijn arm',
       'doet zeer als ik mn hoofd beweeg'], dtype=object)

## **Model Building: GPT-4**

### **A. Data & Model Preparation**

In [ ]:
# Load the API key from an environment variable
openai_api_key = os.getenv('OPENAI_API_KEY')
client = openai.ChatCompletion.create(api_key=openai_api_key)



### **C. Model Development and Experimentation**

### **D. Train model**

### **E. Serialize model**